In [1]:
import pandas as pd
import math as m
import numpy as np
import matplotlib.pyplot as plt
from time import time

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler


Bad key "“backend" on line 1 in
/Users/rfdanti/.matplotlib/matplotlibrc.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.2.1/matplotlibrc.template
or from the matplotlib source distribution


In [7]:
trainset = pd.read_csv("full_trainset_v2.csv")
testset = pd.read_csv("test_merged0.5.csv")

testset.sort_values(['Store', 'Date'], ascending=[True, True], inplace=True)
testset = testset.reset_index(drop=True)
testset = testset.drop(columns='NewPromo2Month', axis=1)
testset = testset.rename(columns={"competition_age_days": "CompetitionAge", "affectedbycompetition": "AffectedByCompetition",
                        "affectedbypromo2": "AffectedByPromo2", "month": "Month"})
testset.head()
trainset.head()

testset["WeekOfYear"] = np.nan
testset["DateOfMonth"] = np.nan
testset['Dates'] = pd.to_datetime(testset['Date'])
testset['DateOfMonth'] = testset['Dates'].dt.day
import datetime
testset['WeekOfYear'] = testset['Dates'].dt.week
testset = testset.drop(columns='Dates', axis=1)
testset['CompetitionDistance'] = trainset['CompetitionDistance'].astype('int')
testset['Customers'] = np.nan
testset.head()

trainset = trainset.drop(["CompetitionOpenSinceMonth", "CompetitionOpenSinceYear", "CompetitionOpenSinceDate",
                          "Promo2SinceWeek", "Promo2SinceYear", "Promo2SinceDate"], axis = 1)
trainset.head()

trainset = trainset[["Date", "Store", "StoreType", "Assortment",
                     "StateHoliday", "SchoolHoliday", "Promo", "Open", "OpeningType",
                     "DayOfWeek", "DateOfMonth", "Month", "WeekOfYear",
                     "Promo2", "PromoInterval", "AffectedByPromo2",
                     "CompetitionDistance", "CompetitionAge", "AffectedByCompetition",
                     "Sales", "Customers"]]
testset = testset[["Date", "Store", "StoreType", "Assortment",
                     "StateHoliday", "SchoolHoliday", "Promo", "Open", "OpeningType",
                     "DayOfWeek", "DateOfMonth", "Month", "WeekOfYear",
                     "Promo2", "PromoInterval", "AffectedByPromo2",
                     "CompetitionDistance", "CompetitionAge", "AffectedByCompetition",
                     "Sales", "Customers"]]             
                   
trainset.info()
testset.info()

#trainset.to_csv("final_trainset.csv", index=False)
#testset.to_csv("final_testset.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050330 entries, 0 to 1050329
Data columns (total 21 columns):
 #   Column                 Non-Null Count    Dtype 
---  ------                 --------------    ----- 
 0   Date                   1050330 non-null  object
 1   Store                  1050330 non-null  int64 
 2   StoreType              1050330 non-null  int64 
 3   Assortment             1050330 non-null  int64 
 4   StateHoliday           1050330 non-null  int64 
 5   SchoolHoliday          1050330 non-null  int64 
 6   Promo                  1050330 non-null  int64 
 7   Open                   1050330 non-null  int64 
 8   OpeningType            1050330 non-null  int64 
 9   DayOfWeek              1050330 non-null  int64 
 10  DateOfMonth            1050330 non-null  int64 
 11  Month                  1050330 non-null  int64 
 12  WeekOfYear             1050330 non-null  int64 
 13  Promo2                 1050330 non-null  int64 
 14  PromoInterval          1050330 non

In [2]:
testset = pd.read_csv("test_merged0.5.csv")
testset

,Store,StoreType,Assortment,Promo2,PromoInterval,DayOfWeek,Date,Sales,Open,Promo,StateHoliday,SchoolHoliday,affectedbypromo2,affectedbycompetition,competition_age_days,month,NewPromo2Month,OpeningType,CompetitionDistance
0,1,2,0,False,0,4,2015-09-17,NaN,True,True,0,False,False,True,2572,9,False,0,1270.0
1,1,2,0,False,0,3,2015-09-16,NaN,True,True,0,False,False,True,2571,9,False,0,1270.0
2,1,2,0,False,0,2,2015-09-15,NaN,True,True,0,False,False,True,2570,9,False,0,1270.0
3,1,2,0,False,0,1,2015-09-14,NaN,True,True,0,False,False,True,2569,9,False,0,1270.0
4,1,2,0,False,0,7,2015-09-13,NaN,False,False,0,False,False,True,2568,9,False,0,1270.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41083,1115,3,2,True,3,3,2015-05-08,NaN,True,True,0,True,True,True,1830,8,False,0,5350.0
41084,1115,3,2,True,3,2,2015-04-08,NaN,True,True,0,True,True,True,1829,8,False,0,5350.0
41085,1115,3,2,True,3,1,2015-03-08,NaN,True,True,0,True,True,True,1828,8,False,0,5350.0
41086,1115,3,2,True,3,7,2015-02-08,NaN,False,False,0,True,True,True,1827,8,False,0,5350.0


In [3]:
testset[testset["Store"]==1]

,Store,StoreType,Assortment,Promo2,PromoInterval,DayOfWeek,Date,Sales,Open,Promo,StateHoliday,SchoolHoliday,affectedbypromo2,affectedbycompetition,competition_age_days,month,NewPromo2Month,OpeningType,CompetitionDistance
0,1,2,0,False,0,4,2015-09-17,NaN,True,True,0,False,False,True,2572,9,False,0,1270.0
1,1,2,0,False,0,3,2015-09-16,NaN,True,True,0,False,False,True,2571,9,False,0,1270.0
2,1,2,0,False,0,2,2015-09-15,NaN,True,True,0,False,False,True,2570,9,False,0,1270.0
3,1,2,0,False,0,1,2015-09-14,NaN,True,True,0,False,False,True,2569,9,False,0,1270.0
4,1,2,0,False,0,7,2015-09-13,NaN,False,False,0,False,False,True,2568,9,False,0,1270.0
5,1,2,0,False,0,6,2015-12-09,NaN,True,False,0,False,False,True,2567,9,False,0,1270.0
6,1,2,0,False,0,5,2015-11-09,NaN,True,False,0,False,False,True,2566,9,False,0,1270.0
7,1,2,0,False,0,4,2015-10-09,NaN,True,False,0,False,False,True,2565,9,False,0,1270.0
8,1,2,0,False,0,3,2015-09-09,NaN,True,False,0,False,False,True,2564,9,False,0,1270.0
9,1,2,0,False,0,2,2015-08-09,NaN,True,False,0,False,False,True,2563,9,False,0,1270.0


In [4]:
testset["Date"] = pd.to_datetime(testset["Date"].astype(str), format = '%Y/%m/%d')
testset[testset["Store"]==1]

,Store,StoreType,Assortment,Promo2,PromoInterval,DayOfWeek,Date,Sales,Open,Promo,StateHoliday,SchoolHoliday,affectedbypromo2,affectedbycompetition,competition_age_days,month,NewPromo2Month,OpeningType,CompetitionDistance
0,1,2,0,False,0,4,2015-09-17,NaN,True,True,0,False,False,True,2572,9,False,0,1270.0
1,1,2,0,False,0,3,2015-09-16,NaN,True,True,0,False,False,True,2571,9,False,0,1270.0
2,1,2,0,False,0,2,2015-09-15,NaN,True,True,0,False,False,True,2570,9,False,0,1270.0
3,1,2,0,False,0,1,2015-09-14,NaN,True,True,0,False,False,True,2569,9,False,0,1270.0
4,1,2,0,False,0,7,2015-09-13,NaN,False,False,0,False,False,True,2568,9,False,0,1270.0
5,1,2,0,False,0,6,2015-12-09,NaN,True,False,0,False,False,True,2567,9,False,0,1270.0
6,1,2,0,False,0,5,2015-11-09,NaN,True,False,0,False,False,True,2566,9,False,0,1270.0
7,1,2,0,False,0,4,2015-10-09,NaN,True,False,0,False,False,True,2565,9,False,0,1270.0
8,1,2,0,False,0,3,2015-09-09,NaN,True,False,0,False,False,True,2564,9,False,0,1270.0
9,1,2,0,False,0,2,2015-08-09,NaN,True,False,0,False,False,True,2563,9,False,0,1270.0


In [5]:
trainset = pd.read_csv("full_trainset_v2.csv")

In [20]:
trainset[trainset["Month"]==9][trainset["Store"]==9]

/Users/rfdanti/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,Date,Store,StoreType,Assortment,StateHoliday,SchoolHoliday,Promo,Open,OpeningType,DayOfWeek,...,Month,WeekOfYear,Promo2,PromoInterval,AffectedByPromo2,CompetitionDistance,CompetitionAge,AffectedByCompetition,Sales,Customers
7779,2013-09-01,9,0,2,0,0,0,0,0,7,...,9,35,0,0,0,2030,4779,1,0,0
7780,2013-09-02,9,0,2,0,1,0,1,0,1,...,9,36,0,0,0,2030,4780,1,6509,589
7781,2013-09-03,9,0,2,0,1,0,1,0,2,...,9,36,0,0,0,2030,4781,1,4986,524
7782,2013-09-04,9,0,2,0,0,0,1,0,3,...,9,36,0,0,0,2030,4782,1,5472,535
7783,2013-09-05,9,0,2,0,0,0,1,0,4,...,9,36,0,0,0,2030,4783,1,4588,466
7784,2013-09-06,9,0,2,0,0,0,1,0,5,...,9,36,0,0,0,2030,4784,1,5272,481
7785,2013-09-07,9,0,2,0,0,0,1,0,6,...,9,36,0,0,0,2030,4785,1,5352,500
7786,2013-09-08,9,0,2,0,0,0,0,0,7,...,9,36,0,0,0,2030,4786,1,0,0
7787,2013-09-09,9,0,2,0,0,1,1,0,1,...,9,37,0,0,0,2030,4787,1,9247,744
7788,2013-09-10,9,0,2,0,0,1,1,0,2,...,9,37,0,0,0,2030,4788,1,6218,516


In [21]:
testset[testset["Month"]==9][testset["Store"]==9]

/Users/rfdanti/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,Date,Store,StoreType,Assortment,StateHoliday,SchoolHoliday,Promo,Open,OpeningType,DayOfWeek,...,Month,WeekOfYear,Promo2,PromoInterval,AffectedByPromo2,CompetitionDistance,CompetitionAge,AffectedByCompetition,Sales,Customers
193,2015-01-09,9,0,2,0,False,True,True,0,2,...,9,2,False,0,False,1270,5509,True,NaN,NaN
195,2015-02-09,9,0,2,0,False,True,True,0,3,...,9,7,False,0,False,1270,5510,True,NaN,NaN
197,2015-03-09,9,0,2,0,False,True,True,0,4,...,9,11,False,0,False,1270,5511,True,NaN,NaN
199,2015-04-09,9,0,2,0,False,True,True,0,5,...,9,15,False,0,False,1270,5512,True,NaN,NaN
201,2015-05-09,9,0,2,0,False,False,True,0,6,...,9,19,False,0,False,1270,5513,True,NaN,NaN
203,2015-06-09,9,0,2,0,False,False,False,0,7,...,9,24,False,0,False,1270,5514,True,NaN,NaN
205,2015-07-09,9,0,2,0,False,False,True,0,1,...,9,28,False,0,False,1270,5515,True,NaN,NaN
207,2015-08-09,9,0,2,0,False,False,True,0,2,...,9,32,False,0,False,1270,5516,True,NaN,NaN
228,2015-09-09,9,0,2,0,False,False,True,0,3,...,9,37,False,0,False,1270,5517,True,NaN,NaN
229,2015-09-13,9,0,2,0,False,False,False,0,7,...,9,37,False,0,False,1270,5521,True,NaN,NaN
